<a href="https://colab.research.google.com/github/englissi/englissi/blob/main/Listening_comprehension_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Listening comprehension test

In [11]:
%%capture
!pip install gradio

In [13]:
%%capture
# First, install the gTTS library if you haven't already
!pip install gTTS

# Import the gTTS library
from gtts import gTTS


In [19]:
import gradio as gr
import requests
from gtts import gTTS
import os

# Fetch the story from the URL
story_url = "https://raw.githubusercontent.com/MK316/Spring2024/main/DLTESOL/project/story02.txt"
story = requests.get(story_url).text

# Define the listening comprehension questions
questions = [
    {
        "question": "1. What inspired many adventurous tales among the villagers of Echo Ridge?",
        "options": ["The towering peaks", "The Guardian of the Glen", "The ancient maps", "The village lore"],
        "answer": "The Guardian of the Glen"
    },
    {
        "question": "2. What did Alex stumble upon in the local library?",
        "options": ["A hidden door", "An ancient map", "A magical book", "A forgotten diary"],
        "answer": "An ancient map"
    },
    {
        "question": "3. Who accompanied Alex on his adventure to find the treasure?",
        "options": ["His siblings", "His best friends Mia and Sam", "His dog", "His teacher"],
        "answer": "His best friends Mia and Sam"
    },
    {
        "question": "4. What did Alex and his friends find inside the ancient chest?",
        "options": ["Gold and jewels", "Old artifacts and a carved eagle statuette", "A magical spell book", "A map to another treasure"],
        "answer": "Old artifacts and a carved eagle statuette"
    },
    {
        "question": "5. How did the village react to the children's discovery?",
        "options": ["They ignored it", "They celebrated and hailed the children as heroes", "They were frightened", "They wanted to keep the artifacts"],
        "answer": "They celebrated and hailed the children as heroes"
    },
    {
        "question": "6. What did the legend of the Guardian of the Glen come to symbolize?",
        "options": ["Wealth and prosperity", "Danger and fear", "Adventure and discovery", "Peace and tranquility"],
        "answer": "Adventure and discovery"
    }
]

# Function to convert text to speech and save as an audio file
def text_to_speech(text, filename):
    tts = gTTS(text)
    tts.save(filename)
    return filename

# Generate audio files for questions
for i, question in enumerate(questions):
    audio_filename = f"question_{i+1}.mp3"
    question["audio"] = text_to_speech(question["question"], audio_filename)

# Function to handle the quiz
def quiz(question_index, user_answer, score, incorrect):
    if question_index >= 0:
        question = questions[question_index]
        if user_answer == question["answer"]:
            score += 1
        else:
            incorrect.append(f"Question {question_index + 1}: {question['question']}\nCorrect answer: {question['answer']}")
        question_index += 1

    if question_index < len(questions):
        return (
            question_index,
            score,
            incorrect,
            gr.update(value=questions[question_index]["audio"], visible=True),
            gr.update(choices=questions[question_index]["options"], value=None, visible=True),
            gr.update(visible=True),
            gr.update(visible=False)
        )
    else:
        results = "\n".join(incorrect)
        return (
            -1,
            score,
            incorrect,
            gr.update(visible=False),
            gr.update(visible=False),
            gr.update(visible=False),
            gr.update(visible=True, value=f"Your score: {score}\n\n{results}")
        )

# Prepare the initial question audio and options
initial_audio = questions[0]["audio"]
initial_options = questions[0]["options"]

# Create the Gradio interface
with gr.Blocks() as iface:
    gr.Markdown("## The Guardian's Secret")

    question_index = gr.State(0)
    score = gr.State(0)
    incorrect = gr.State([])

    audio_output = gr.Audio(value=initial_audio, autoplay=False, label="")
    options = gr.Radio(choices=initial_options, label="Your Answer")
    next_button = gr.Button("Next")

    final_score = gr.Textbox(label="Final Score", visible=False)

    next_button.click(
        quiz,
        [question_index, options, score, incorrect],
        [question_index, score, incorrect, audio_output, options, next_button, final_score]
    )

# Launch the interface
iface.launch(share=True)





Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://47bab5f61e5f820be8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
